In [1]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [1]:
sys.path

NameError: name 'sys' is not defined

In [3]:
import os
import json

import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM as LLaMAForCausalLM
from transformers import LlamaTokenizer as LLaMATokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model 
from datasets import load_dataset
from EvaluateTestSet import EvaluateTestSet
from transformers.integrations import TensorBoardCallback
from transformers import GenerationConfig



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)


/root/HSE_diploma/prompter/templates/


In [4]:

def init_lora_model_and_tokenizer(default_model,
                             LORA_R,
                             LORA_ALPHA,
                             LORA_DROPOUT
                            ):


    """
        
    """
    model = LLaMAForCausalLM.from_pretrained(
    default_model,
    load_in_8bit=True,
    device_map="auto",
    )
    tokenizer = LLaMATokenizer.from_pretrained(
        default_model, add_eos_token=True
    )

    model = prepare_model_for_int8_training(model)

    config = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )

    model = get_peft_model(model, config)

    tokenizer.pad_token_id = 0  # unk. we want this to be different from the eos token

    return model, tokenizer

In [5]:
CONFIG_PATH = "/root/HSE_diploma/experiments_configs/"
EXPERIMENTS_PATH = "/root/experiments/"
experiment_name = "t2c_concode_220428_v38_plustwoepoch"
# t2c_concode_220428_v18.json

In [6]:
# !nvidia-smi

In [7]:
!ls {current_config_path}

ls: cannot access '{current_config_path}': No such file or directory


In [8]:
# !ls /root/HSE_diploma/experiments_configs/t2c_concode_220428_t2c_plustwoepoch_config.json

In [9]:
current_config_path = os.path.join(CONFIG_PATH, experiment_name + "_config.json")
experiment_config = json.load(open(current_config_path, "r"))

assert experiment_config['experiment_name'] == experiment_name

In [10]:
# !ls /root/HSE_diploma/experiments_configs/t2c_concode_220428_plustwoepoch_config.json
# !ls /root/HSE_diploma/experiments_configs/t2c_concode_220428_v38_plustwoepoch_config.json
# t2c_concode_220428_v38_plustwoepoch_config.json

In [11]:
# experiment_config['resume_from_checkpoint'] = True

In [12]:
assert experiment_config['resume_from_checkpoint'] == False

AssertionError: 

In [13]:
# del 
# experiment_name = "t2c_concode_220428_v38_plusoneepoch"
# experiment_config['experiment_name'] = "t2c_concode_220428_v38_plusoneepoch"

In [14]:
assert experiment_config['experiment_name'] == experiment_name

In [15]:
current_experiment_path = os.path.join(EXPERIMENTS_PATH, experiment_name)

In [16]:
print("current_experiment_path", current_experiment_path)

current_experiment_path /root/experiments/t2c_concode_220428_v38_plustwoepoch


In [17]:
!mkdir {current_experiment_path}

mkdir: cannot create directory ‘/root/experiments/t2c_concode_220428_v38_plustwoepoch’: File exists


In [18]:
!nvidia-smi

Wed May 24 21:09:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:41:00.0 Off |                  N/A |
| 30%   26C    P8    14W / 350W |      3MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
experiment_config["EPOCHS"] = 4

In [20]:
json.dump(experiment_config, open(current_experiment_path + \
                                  "/experiment_config.json", 
                                  "w+"
                                 )
         )

In [21]:
model, tokenizer = init_lora_model_and_tokenizer(default_model = experiment_config["default_model"],
                                                 LORA_R = experiment_config["LORA_R"],
                                                 LORA_ALPHA = experiment_config["LORA_ALPHA"],
                                                 LORA_DROPOUT = experiment_config["LORA_DROPOUT"]
                                                )



Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
data = load_dataset("json", 
                    data_files = {"train": experiment_config["fn_train_dataset"],
                                  "eval":  experiment_config["fn_eval_dataset"]
                                 }
                   )

Found cached dataset json (/root/.cache/huggingface/datasets/json/default-02cbc528b265cbf4/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

# tokenizer.pad_token_id 0
# tokenizer.eos_token_id 2
# tokenizer.bos_token_id 1
# tokenizer.eos_token_id 2
# model.config.pad_token_id 0
# model.config.eos_token_id 2
# model.config.bos_token_id 1
# model.config.eos_token_id 2
# trainable params: 805306368 || all params: 7543721984 || trainable%: 10.675186197317846


tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 805306368 || all params: 7543721984 || trainable%: 10.675186197317846


In [24]:
experiment_config

{'experiment_name': 't2c_concode_220428_v38_plustwoepoch',
 'fn_train_dataset': '/root/data/t2c_train.json',
 'fn_eval_dataset': '/root/data/t2c_answers.json',
 'default_model': 'yahma/llama-7b-hf',
 'MICRO_BATCH_SIZE': 2,
 'BATCH_SIZE': 10,
 'EPOCHS': 4,
 'LEARNING_RATE': 0.0002,
 'CUTOFF_LEN': 512,
 'LORA_R': 1536,
 'LORA_ALPHA': 1536,
 'LORA_DROPOUT': 0.05,
 'warmup_steps': 200,
 'fp16': True,
 'logging_steps': 10,
 'eval_steps': 100,
 'evaluation_strategy': 'steps',
 'save_total_limit': 1,
 'save_strategy': 'steps',
 'save_steps': 500,
 'seed': 42,
 'logging_strategy': 'steps',
 'report_to': 'tensorboard',
 'mlm': False,
 'truncation': True,
 'padding': 'max_length',
 'config_use_cache': False,
 'resume_from_checkpoint': True,
 'bleu_batch_size': 5,
 'GRADIENT_ACCUMULATION_STEPS': 5,
 'log_bleu_steps_factor': 50,
 'load_best_model_at_end': False}

In [25]:
from prompter import Prompter
prompter = Prompter()

def generate_prompt(data_point):
    if "input" in data_point and data_point["input"]:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        input = data_point["input"],
                                        label = data_point["output"]
                                       )
    else:
        return prompter.generate_prompt(instruction = data_point["instruction"],
                                        #input = None,
                                        label = data_point["output"]
                                       )

/root/HSE_diploma/prompter/templates/


In [26]:
experiment_config["truncation"],\
experiment_config["CUTOFF_LEN"],\
experiment_config["padding"]

# (True, 256, 'max_length')

(True, 512, 'max_length')

In [27]:
data = data.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=experiment_config["truncation"],
        max_length=experiment_config["CUTOFF_LEN"],
        padding=experiment_config["padding"]
    )
)


Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
print(current_experiment_path)

/root/experiments/t2c_concode_220428_v38_plustwoepoch


In [29]:
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=data["train"],
    eval_dataset=data['eval'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=experiment_config["MICRO_BATCH_SIZE"],
        gradient_accumulation_steps=experiment_config["GRADIENT_ACCUMULATION_STEPS"],
        warmup_steps=experiment_config["warmup_steps"],
        num_train_epochs=experiment_config["EPOCHS"],
        learning_rate=experiment_config["LEARNING_RATE"],
        fp16=experiment_config["fp16"],
        logging_steps=experiment_config["logging_steps"],        
        evaluation_strategy = experiment_config['evaluation_strategy'],
        eval_steps=experiment_config["eval_steps"],
        output_dir=current_experiment_path,#"lora-alpaca",
        save_total_limit=experiment_config["save_total_limit"],
        save_strategy = experiment_config["save_strategy"],
        
        save_steps = experiment_config["save_steps"],
        seed=experiment_config["seed"],
        logging_dir=current_experiment_path,
        logging_strategy=experiment_config["logging_strategy"],
        report_to=experiment_config["report_to"],
        load_best_model_at_end = experiment_config["load_best_model_at_end"]
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, 
                                                               mlm=experiment_config["mlm"]
                                                              ),
#     callbacks = [MyCustomCallback]
)
model.config.use_cache = experiment_config["config_use_cache"]
# print(len(trainer.optimizer.state['found_inf_per_device']))


trainer.train(resume_from_checkpoint=experiment_config["resume_from_checkpoint"])

model.save_pretrained(current_experiment_path)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
22100,5.199100,5.260817
22200,3.723000,4.042470
22300,3.427400,3.636251
22400,3.069600,3.442064
22500,3.092900,3.318936
22600,2.884900,3.237265
22700,2.826200,3.134555
22800,2.879900,3.075089
22900,2.817600,3.053328
23000,2.703100,3.016374


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

In [33]:
model.save_pretrained(current_experiment_path)

In [34]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay         118G  102G   17G  87% /


In [32]:
# print(current_experiment_path)
# !ls -lah /root/experiments/

total 24K
drwxr-xr-x  9 root root  249 May 24 20:47 .
drwx------  1 root root 4.0K May 19 17:59 ..
drwxr-xr-x  7 root root 4.0K May 19 19:33 t2c_concode_220428_v34
drwxr-xr-x 10 root root 4.0K May 19 21:39 t2c_concode_220428_v35_prefix
drwxr-xr-x 23 root root 4.0K May 20 00:24 t2c_concode_220428_v36
drwxr-xr-x  4 root root  196 May 21 19:36 t2c_concode_220428_v37
drwxr-xr-x  5 root root  280 May 23 20:27 t2c_concode_220428_v38
drwxr-xr-x  7 root root 4.0K May 24 01:41 t2c_concode_220428_v38_plusoneepoch
drwxr-xr-x  8 root root 4.0K May 25 06:38 t2c_concode_220428_v38_plustwoepoch


In [30]:
# 0

0

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
20100,5.525600,5.543083
20200,4.801400,5.058121
20300,4.039300,4.385653
20400,3.508500,3.822434
20500,3.395900,3.564281
20600,3.145800,3.427295
20700,3.021800,3.334805
20800,2.976300,3.256720
20900,2.793700,3.187501
21000,2.928900,3.132894


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

OutOfMemoryError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 23.69 GiB total capacity; 20.26 GiB already allocated; 124.06 MiB free; 22.50 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [44]:
model.save_pretrained(current_experiment_path)

In [46]:
print(current_experiment_path)
!ls {current_experiment_path}

/root/experiments/t2c_concode_220428_v38_plusoneepoch
1684702442.0964742   checkpoint-22000
1684702685.6640449   events.out.tfevents.1684702442.c71a00d70d2c.3733.0
1684875698.2082248   events.out.tfevents.1684702685.c71a00d70d2c.3814.0
1684875738.6857948   events.out.tfevents.1684875698.c71a00d70d2c.4208.0
adapter_config.json  events.out.tfevents.1684875738.c71a00d70d2c.4208.2
adapter_model.bin    experiment_config.json


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.026000,1.084104
200,1.353800,7.147864
300,0.936000,1.097126
400,0.952800,1.089456
500,0.880900,1.091858
600,0.957200,1.095044
700,0.945000,1.096369
800,0.903600,1.099093
900,0.994400,1.098762
1000,0.844600,1.111839


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.170200,1.188443
200,0.980600,1.100288
300,0.954200,1.077399
400,0.889600,1.070261
500,0.972500,1.061411
600,0.911500,1.059370
700,0.949100,1.056929
800,0.905200,1.055107
900,0.903500,1.053152
1000,0.891000,1.052750


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during q

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,1.179400,1.262794
200,1.115300,1.161276
300,0.991500,1.134106
400,1.003000,1.125500
500,0.930900,1.119192
600,0.972400,1.115012
700,1.053000,1.112176
800,0.965700,1.112523
900,0.982100,1.108011


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



...

{'EM': 0.0, 'BLEU': 7.699406180106954e-06, 'brevity_penalty': 0.0019735783221064072, 'ratio': 0.13835263835263836, 'translation_length': 430, 'reference_length': 3108, 'precisions_0': 0.002320185614849188, 'precisions_1': 0.0030211480362537764, 'precisions_2': 0.004329004329004329, 'precisions_3': 0.007633587786259542}


/opt/conda/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


generation_config: GenerationConfig {
  "max_new_tokens": 30,
  "transformers_version": "4.29.2"
}



 69%|████████████████████████████▉             | 69/100 [04:36<02:04,  4.01s/it]


KeyboardInterrupt: 

In [26]:
!df -h .

Filesystem      Size  Used Avail Use% Mounted on
overlay         118G   64G   55G  54% /


In [27]:
!ls -lah {current_experiment_path}

total 3.1G
drwxr-xr-x 5 root root  280 May 23 20:27 .
drwxr-xr-x 7 root root  163 May 21 20:50 ..
drwxr-xr-x 2 root root   64 May 21 20:54 1684702442.0964742
drwxr-xr-x 2 root root   64 May 21 20:58 1684702685.6640449
-rw-r--r-- 1 root root  343 May 23 20:27 adapter_config.json
-rw-r--r-- 1 root root 3.1G May 23 20:27 adapter_model.bin
drwxr-xr-x 2 root root  243 May 23 20:27 checkpoint-20000
-rw-r--r-- 1 root root 4.1K May 21 20:54 events.out.tfevents.1684702442.c71a00d70d2c.3733.0
-rw-r--r-- 1 root root 366K May 23 20:27 events.out.tfevents.1684702685.c71a00d70d2c.3814.0
-rw-r--r-- 1 root root  819 May 21 20:55 experiment_config.json


In [28]:
current_experiment_path

'/root/experiments/t2c_concode_220428_v38'

In [29]:
model.save_pretrained(current_experiment_path)

In [3]:
!ls /root/experiments/t2c_concode_220428_v34

1684518421.931266    checkpoint-500
1684518421.9377556   events.out.tfevents.1684518421.c71a00d70d2c.1489.0
1684518911.7628372   events.out.tfevents.1684518421.c71a00d70d2c.1489.2
1684518911.768738    events.out.tfevents.1684518911.c71a00d70d2c.1622.0
adapter_config.json  events.out.tfevents.1684518911.c71a00d70d2c.1622.2
adapter_model.bin    experiment_config.json
